In [1]:
import polars as pl
import pandas as pd
import csv
import json
import os

In [2]:
train_path='/home/ming/GraduateProject/Data/MINDlarge_train'
valid_path='/home/ming/GraduateProject/Data/MINDlarge_dev'
paths=[train_path,valid_path]

In [3]:
import json

def transform(path):
    data = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            cols = line.rstrip("\n").split("\t")
            if len(cols) != 8:
                continue  # 跳过异常行

            news_id, category, subcat, title, abstract, url, title_ent, abstract_ent = cols

            data.append({
                "news_id": news_id,
                "category": category,
                "subcategory": subcat,
                "title": title,
                "abstract": abstract,
                "url": url,
                "title_entities": json.loads(title_ent),
                "abstract_entities": json.loads(abstract_ent),
            })
    return data

In [4]:
def save(path,data):
    with open(path, "w", encoding="utf-8", newline="") as f:
        writer = csv.DictWriter(
            f,
            fieldnames=[
                "news_id",
                "category",
                "subcategory",
                "title",
                "abstract",
                "url",
                "title_entities",
                "abstract_entities",
            ]
        )
        writer.writeheader()

        for row in data:
            row = row.copy()
            row["title_entities"] = json.dumps(row["title_entities"], ensure_ascii=False)
            row["abstract_entities"] = json.dumps(row["abstract_entities"], ensure_ascii=False)
            writer.writerow(row)


In [5]:
def save_parquet(path, data):
    df = pl.from_dicts(data)
    df.write_parquet(path)

In [7]:
for path in paths:
    data=transform(os.path.join(path,'news.txt'))
    save_parquet(os.path.join(path,'news.parquet'),data)